In [61]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

In [62]:
def get_image_paths(base_dir='/Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive'):
    """Lấy đường dẫn của tất cả ảnh"""
    brands = {
        'adidas_final': ['ao_adidas', 'quan_adidas'],
        'uniqlo_final': ['ao_uniqlo', 'quan_uniqlo'],
        'yame_final': ['ao_yame', 'quan_yame'],
        'nike_final': ['ao_nike', 'quan_nike'],
        'icondenim_final': ['ao_icondenim', 'quan_icondenim']
    }
    
    image_paths = []
    image_labels = []
    
    print("\n=== LOADING IMAGES ===")
    for brand, categories in brands.items():
        brand_path = os.path.join(base_dir, brand)
        print(f"\nKiểm tra thư mục: {brand_path}")
        
        if os.path.exists(brand_path):
            for category in categories:
                category_path = os.path.join(brand_path, category)
                print(f"  Kiểm tra thư mục con: {category_path}")
                
                if os.path.exists(category_path):
                    images = [f for f in os.listdir(category_path) 
                            if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                    print(f"    Tìm thấy {len(images)} ảnh")
                    
                    for img_name in images:
                        img_path = os.path.join(category_path, img_name)
                        image_paths.append(img_path)
                        image_labels.append(category)
                else:
                    print(f"    Thư mục không tồn tại: {category_path}")
        else:
            print(f"  Thư mục không tồn tại: {brand_path}")
    
    print(f"\nTổng số ảnh tìm thấy: {len(image_paths)}")
    print("Danh sách các nhãn và số lượng:")
    for label in set(image_labels):
        count = image_labels.count(label)
        print(f"- {label}: {count} ảnh")
    
    return image_paths, image_labels

def create_data_generators(base_dir='/Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive', batch_size=128, img_size=(224, 224), validation_split=0.2):
    """Tạo data generators cho training và validation"""
    print("\n=== CREATING DATA GENERATORS ===")
    print(f"Batch size: {batch_size}")
    print(f"Image size: {img_size}")
    print(f"Validation split: {validation_split}")
    
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,  # Chỉ giữ lại augmentation cần thiết
        validation_split=validation_split
    )
    
    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=validation_split
    )
    
    train_generator = train_datagen.flow_from_directory(
        base_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    
    val_generator = val_datagen.flow_from_directory(
        base_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=True
    )
    
    print("\nThông tin về data generators:")
    print(f"Số lượng classes: {len(train_generator.class_indices)}")
    print("Class mapping:")
    for class_name, class_idx in train_generator.class_indices.items():
        print(f"- {class_name}: {class_idx}")
    
    return train_generator, val_generator

image_paths, image_labels = get_image_paths()


=== LOADING IMAGES ===

Kiểm tra thư mục: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/adidas_final
  Kiểm tra thư mục con: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/adidas_final/ao_adidas
    Tìm thấy 8397 ảnh
  Kiểm tra thư mục con: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/adidas_final/quan_adidas
    Tìm thấy 5148 ảnh

Kiểm tra thư mục: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/uniqlo_final
  Kiểm tra thư mục con: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/uniqlo_final/ao_uniqlo
    Tìm thấy 924 ảnh
  Kiểm tra thư mục con: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/uniqlo_final/quan_uniqlo
    Tìm thấy 398 ảnh

Kiểm tra thư mục: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/yame_final
  Kiểm tra thư mục con: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/yame_final/ao_yame
    Tìm thấy 2120 ảnh
  Kiểm tra thư mục con: /Users/tvq/Documents/Computer Vision/BÁO CÁO CK/archive/yame_final/

In [65]:
def create_model(num_classes, img_size=(224, 224)):
    """Tạo mô hình MobileNetV2"""
    print("\n=== CREATING MODEL ===")
    print(f"Number of classes: {num_classes}")
    print(f"Input image size: {img_size}")
    
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(*img_size, 3)
    )
    
    base_model.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    print("\nModel Summary:")
    model.summary()
    
    return model
train_generator, val_generator = create_data_generators()



=== CREATING DATA GENERATORS ===
Batch size: 128
Image size: (224, 224)
Validation split: 0.2
Found 23768 images belonging to 5 classes.
Found 5939 images belonging to 5 classes.

Thông tin về data generators:
Số lượng classes: 5
Class mapping:
- adidas_final: 0
- icondenim_final: 1
- nike_final: 2
- uniqlo_final: 3
- yame_final: 4


In [66]:
def compile_model(model, learning_rate=0.001):
    """Compile model"""
    print("\n=== COMPILING MODEL ===")
    print(f"Learning rate: {learning_rate}")
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Model compiled successfully!")
    return model
model = compile_model(model)



def train_model(model, train_generator, val_generator, epochs=20, save_path='best_model.h5'):
    """Huấn luyện mô hình"""
    print("\n=== TRAINING MODEL ===")
    print(f"Number of epochs: {epochs}")
    print(f"Save path: {save_path}")
    
    callbacks = [
        ModelCheckpoint(
            save_path,
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-6
        )
    ]
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=callbacks
    )
    
    print("\nTraining completed!")
    print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
    print(f"Best validation loss: {min(history.history['val_loss']):.4f}")
    
    return history

history = train_model(model, train_generator, val_generator)




=== COMPILING MODEL ===
Learning rate: 0.001
Model compiled successfully!

=== TRAINING MODEL ===
Number of epochs: 20
Save path: best_model.h5
Epoch 1/20


2025-05-28 14:58:25.743197: W tensorflow/core/framework/op_kernel.cc:1844] UNKNOWN: UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x32631f310>
Traceback (most recent call last):

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 248, in _finite_generator
    yield self._standardize_batch(self.py_dataset[i])

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/keras/src/le

  1/186 ━━━━━━━━━━━━━━━━━━━━ 21:25 7s/step - accuracy: 0.6797 - loss: 0.7478

2025-05-28 14:58:26.196637: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: UNKNOWN: UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x32631f310>
Traceback (most recent call last):

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 248, in _finite_generator
    yield self._standardize_batch(self.py_dataset[i])

  File "/Users/tvq/Library/

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x32631f310>
Traceback (most recent call last):

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/tvq/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 248, in _finite_generator
    yield self._standardize_batch(self.py_dataset[i])

  File "/Users/tvq/Library/Python/

In [43]:
def plot_training_history(history):
    """Vẽ đồ thị quá trình training"""
    print("\n=== PLOTTING TRAINING HISTORY ===")
    
    plt.figure(figsize=(12, 4))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

NameError: name 'history' is not defined

In [42]:
def predict_image(model, image_path, img_size=(224, 224)):
    """Dự đoán một ảnh"""
    print("\n=== PREDICTING IMAGE ===")
    print(f"Image path: {image_path}")
    
    # Load và preprocess ảnh
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    # Dự đoán
    predictions = model.predict(img_array)
    
    print("\nPrediction results:")
    print(f"Shape of predictions: {predictions.shape}")
    print(f"Raw predictions: {predictions[0]}")
    
    return predictions

model = compile_model(model)



=== COMPILING MODEL ===
Learning rate: 0.001
Model compiled successfully!
